# IBM Capstone Project: Segmenting & Clustering Neighborhoods in Toronto pt.2/2

In [8]:
# import necessary libraries
import numpy as np
import pandas as pd
import msgpack
import json
import requests
from bs4 import BeautifulSoup
import csv
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
done=print('done!')
done

done!


In [9]:
# install required libraries
!pip install folium
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge geocoder -y

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

In [10]:
# From pt. 1, create the pandas dataframe
link=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(link.text,'lxml')


# write into a csv file
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
csv_file=open('toronto_1.csv','w')
csv_write=csv.writer(csv_file)
csv_write.writerow(['PostalCode','Borough','Neighborhood'])
for tr in soup.find_all('tr')[1:]:
    td_all = tr.find_all('td')
    if len(td_all) == 3:
        PostalCode=td_all[0].text
        Borough=td_all[1].text
        Neighborhood=td_all[2].text
    csv_write.writerow([PostalCode,Borough,Neighborhood])
csv_file.close()

# assign to a pandas dataframe
df_toronto=pd.read_csv('toronto_1.csv')

## clean the data

# remove \n enters
df_toronto[df_toronto.columns] =df_toronto.apply(lambda x: x.str.strip('\n'))
# remove all rows with 'Not assigned' Boroughs
df_toronto_cleaned = df_toronto[df_toronto.Borough!='Not assigned']
#preview shape
df_toronto_cleaned.shape

(212, 3)

In [11]:
# preview
df_toronto_cleaned.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
# reset index
df_toronto_cleaned=df_toronto_cleaned.reset_index()
df_toronto_cleaned.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


In [13]:
df_toronto_cleaned.drop('index',axis=1,inplace=True)
df_toronto_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Question 2: Get geographical lattitudes & longitudes based on a given postal code

In [23]:
# collect csv file of coordinates
link="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
df_toronto_coordinates=pd.read_csv(link)
df_toronto_coordinates.columns=['PostalCode','Latitude','Longitude']

# merge into a pandas dataframe
df_toronto_coordinates = pd.merge(df_toronto_cleaned, df_toronto_coordinates, on="PostalCode")
df_toronto_coordinates = df_toronto_coordinates[df_toronto_coordinates['Borough'].str.contains('Toronto')].reset_index(drop=True)

# Preview
df_toronto_coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


## Question 3: Explore and cluster the neighborhoods in Toronto.

In [24]:
address = 'Toronto' #apply only to neighborhoods in Toronto
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinates of Toronto are 43.653963, -79.387207


Input credentials (cell hidden below)

In [25]:
# The code was removed by Watson Studio for sharing.

In [27]:
# Toronto folium map
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers
for lat, lng, borough, neighborhood in zip(df_toronto_coordinates['Latitude'],
                                          df_toronto_coordinates['Longitude'],
                                          df_toronto_coordinates['Borough'],
                                          df_toronto_coordinates['Neighborhood']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(toronto_map)

toronto_map